In [4]:
import csv
import re

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

import pandas as pd

from scipy import signal







In [273]:
class read_spectrometer():
    def __init__(self):

        #regex for sample description classification. example file name: sample1-H20-T20-Nikon10x-001
        self.temperature_re = re.compile('T\d+.?\d*')
        self.humidity_re = re.compile('H\d+.?\d*-')
        self.spot_number_re = re.compile('sample\d+-')
        
        
    def convert_matlab_scan(self):
        'this converts matlab data into np.ndarray '
        self.mat_scan_contents = sio.loadmat(self.folder +'\\'+ self.scan_filename)['scan']
        self.scan_result = {}
        # catagorise scan result into spectrum, description and wavelength
        for i in ['spec', 'desc', 'wl']:
            self.scan_result[i] = self.mat_scan_contents[i]
        # total scan numbers
        self.total_scan_number = len(self.scan_result['wl'])
        
        
        def convert_struct_to_column(table, header, scan_header, scan_number):
            'a short function to add column to dict' 
            table[header] = []
            for i in self.scan_result[scan_header][scan_number][0]:
                table[header].append(float(i))
        
        table = {}
        convert_struct_to_column(table, 'wavelength', 'wl', scan_number = 0)
        for i in range(self.total_scan_number):
            # the description about sample temperature and humidity
            description = str(self.scan_result['desc'][i][0]).replace("[","").replace("]","").replace("\'","")
            convert_struct_to_column(table, description, 'spec', scan_number = i)
            
        self.df = pd.DataFrame(table)
        
        # rearrange the header
        header = self.df.columns.tolist()
        header = header[-1:] + header[:-1]
        self.df = self.df[header]
        
        #Save to csv
        self.df.to_csv(self.folder +'\\'+ self.output_filename,  sep=',')

        
    def convert_matlab_scan_2(self):
        'this converts matlab data into np.ndarray '
        self.mat_scan_contents = sio.loadmat(self.folder +'\\'+ self.scan_filename)['scan']
        self.scan_result = {}
        # catagorise scan result into spectrum, description and wavelength
        for i in ['spec', 'desc', 'wl']:
            self.scan_result[i] = self.mat_scan_contents[i]
            
        '''Modify the description to match the size of wavelength'''
        # total spec steps
        self.spec_steps = len(self.scan_result['wl'][0][0])
        # format descriptions to be list of strings
        self.scan_result['formated_desc'] = (list(map(lambda desc: desc[0][0].replace("[","").replace("]","").replace("\'","") , self.scan_result['desc'])))
        
        desc_column = []
        # duplicate elements in desc by number times
        list(map(
            lambda j:list(map(
                lambda k: desc_column.append(k),
                j)), 
            # create a list with repeated elements in desc (in the form of list)
            map(
                lambda i: [i]*self.spec_steps,  
                self.scan_result['formated_desc'])))
        
        wavelength_column = []
        list(map(
            lambda sample: list(map(
                lambda wavelength: wavelength_column.append(wavelength[0]), 
                sample[0])),
                self.scan_result['wl']))
        
        spec_column = []
        list(map(
            lambda sample: list(map(
                lambda spec: spec_column.append(spec[0]), 
                sample[0])),
                self.scan_result['spec']))
      
        table = {}
        table['sample_name'] = desc_column
        table['wavelength'] = wavelength_column
        table['spec'] = spec_column

        
        #Save to csv
        self.df = pd.DataFrame(table)
        self.df.to_csv(self.folder +'\\'+ self.output_filename,  sep=',')

        
        
    
scan = read_spectrometer()
scan.folder = r'D:\GDrive\Research\BIP\Humidity sensor project\data\20170810 - temper'
scan.scan_filename = 'smoothed_peaks.mat'
scan.output_filename = 'output_smoothed_peaks.csv'
scan.convert_matlab_scan_2()

In [216]:
desc = ['ade', 'b', 'c']
number = 5
table = []
# duplicate elements in desc by number times
list(map(
    lambda j:list(map(
        lambda k: table.append(k),
        j)), 
    # create a list with repeated elements in desc (in the form of list)
    map(
        lambda i: [i]*number, 
        desc)))
     
print(table)
    
    

['ade', 'ade', 'ade', 'ade', 'ade', 'b', 'b', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c']
